In [1]:
def split(p1, p2):
    [a, b, c, d] = sorted([*p1, *p2])
    
    if (a == b and b == c and c == d):
        return [(a, a)]
    elif (a == b and b == c):
        return [(a, a), (a+1, d)]
    elif (b == c and c == d):
        return [(a, b-1), (b, b)]
    elif (a == b) and (c == d):
        return [(a, c)]
    elif (a == b):
        return [(a, c), (c+1, d)]
    elif (b == c):
        return [(a, b-1), (b, b), (b+1, d)]
    elif (c == d):
        return [(a, b-1), (b+1, d)]
    else:
        return [(a, b-1), (b, c), (c+1, d)]

In [2]:
import itertools

class Segment:
    def intersection(self, other):
        a, b = self, other
        x1 = max(min(a.x1, a.x2), min(b.x1, b.x2))
        x2 = min(max(a.x1, a.x2), max(b.x1, b.x2))
        if x1 <= x2:
            return type(self)(x1, x2)
    __and__ = intersection

    def difference(self, other):
        inter = self & other
        
        # we've ensured an intersection
        if not inter:
            yield self
            return
        
        x_split = split((self.x1, self.x2), (other.x1, other.x2))
                                   
        for (x1, x2) in x_split:
            rect = type(self)(x1, x2)
            
            if not rect.subset(inter) and rect.subset(self):
                yield rect

    __sub__ = difference

    def __init__(self, x1, x2):
        self.x1, self.x2 = x1, x2

    def __iter__(self):
        yield self.x1
        yield self.x2


    def __eq__(self, other):
        return isinstance(other, Segment) and tuple(self) == tuple(other)
    def __ne__(self, other):
        return not (self == other)

    def __repr__(self):
        return type(self).__name__ + repr(tuple(self))
    
    def volume(self):
        return (self.x2 - self.x1 + 1)
    
    def subset(self, other):
        return self == self & other
    
    def union(self, other):
        # guaranteed no overlap
        inter = self & other
        
        if inter:
            yield inter
        
        yield from self.difference(other)
        yield from other.difference(self)
    __or__ = union

In [47]:
a = Segment(-1, 4)
b = Segment(6, 8)

In [51]:
(a&b)

In [50]:
list(a.difference(b))

[Segment(-1, 4)]

In [52]:
list(b.difference(a))

[Segment(6, 8)]

In [48]:
assert a.volume() - (a&b).volume() == sum(c.volume() for c in a.difference(b))

AttributeError: 'NoneType' object has no attribute 'volume'

In [43]:
assert b.volume() - (a&b).volume() == sum(c.volume() for c in b.difference(a))

In [44]:
assert a.volume() + b.volume() - (a&b).volume() == sum(c.volume() for c in a.union(b))
assert a.volume() + b.volume() - (a&b).volume() == sum(c.volume() for c in b.union(a))

In [45]:
list(a.union(b))

[Segment(4, 4), Segment(-1, 3), Segment(5, 8)]

In [46]:
list(b.union(a))

[Segment(4, 4), Segment(5, 8), Segment(-1, 3)]